
History:
9.13.LAST checkpoint



In [1]:
import numpy as np 


In [2]:
import pandas as pd

In [82]:
print('9.13.20 ') #last update

9.13.20 


In [4]:
df = pd.read_csv('feature_selection_test.csv')
df.head(5) 


,Username,Mobility,ArmSwelling,BreastSwelling,Skin,PAS,FHT,DISCOMFORT,SYM_COUNT,TIME_LAPSE,...,chemo_numbered,Radiation,Age,SLNB_Removed_LN,ALND_Removed_LN,SLNB_ALND_Removed,Mastectomy,Lumpectomy,Hormonal,BMI
0,ML380,2,0,2,0,1,0,1,5,0.8,...,1,1,70.0,1,0,0,0,1,1.0,39.8
1,ML381,1,0,0,1,0,0,1,4,2.2,...,1,1,54.0,2,0,0,0,1,1.0,25.2
2,ML382,0,0,0,0,0,0,0,0,6.3,...,1,1,70.0,0,0,7,0,1,1.0,33.8
3,ML383,0,0,0,0,1,3,3,10,4.3,...,1,1,40.0,0,0,15,0,1,0.0,21.1
4,ML384,0,0,0,0,0,0,0,0,3.1,...,0,1,76.0,1,0,0,0,1,1.0,27.2


PRE-PROCESS


In [86]:
feature_names= ['Username','Mobility','ArmSwelling','BreastSwelling', 'Skin', 'PAS','FHT','DISCOMFORT','SYM_COUNT','TIME_LAPSE','LVC','ChestWallSwelling','fluid_total']
data_df= df[feature_names] #data_df has all the data

In [87]:
data_df.shape

(191, 13)

In [88]:
data_df = data_df[data_df.TIME_LAPSE != 'XXX'] #IGNORE THE ML509 patient

In [89]:
data_df = data_df[data_df.Username != 'ML380'] #the data needs to be ML381 AND ONWARDS 

In [90]:
#pd.to_numeric(data_df['TIME_LAPSE'], downcast="float")
data_df['TIME_LAPSE'] =data_df['TIME_LAPSE'].astype('float')
data_df['LVC'] =data_df['LVC'].astype('float')

data_df['Mobility'] =data_df['Mobility'].astype('int')
data_df['ArmSwelling'] =data_df['ArmSwelling'].astype('int')
data_df['BreastSwelling'] =data_df['BreastSwelling'].astype('int')
data_df['Skin'] =data_df['Skin'].astype('int')
data_df['PAS'] =data_df['PAS'].astype('int')
data_df['FHT'] =data_df['FHT'].astype('int')
data_df['DISCOMFORT'] =data_df['DISCOMFORT'].astype('int')
data_df['SYM_COUNT'] =data_df['SYM_COUNT'].astype('int')
data_df['ChestWallSwelling'] =data_df['ChestWallSwelling'].astype('int')
data_df['fluid_total'] =data_df['fluid_total'].astype('int')

data_df['logged_TIME_LAPSE'] = np.log(data_df['TIME_LAPSE'])

In [91]:
# drop rows containing nan
data_df = data_df.dropna()


In [92]:
data_df= data_df[data_df.TIME_LAPSE >=0.5] #remove patients more than 6months time lapse

Arry to be passed into X model

In [93]:
X_model_df = data_df[['Mobility','ArmSwelling','BreastSwelling', 'Skin', 'PAS','FHT','DISCOMFORT','SYM_COUNT','logged_TIME_LAPSE']]

In [94]:
X_model_df.shape

(175, 9)

In [95]:
X_model_df.isnull().values.any()
# drop rows containing nan


False

In [96]:
'''convert X_model dataframe into array'''
X_model_arr = np.array(X_model_df)
X_model_arr.shape

(175, 9)

In [97]:
import pickle
loaded_clf = pickle.load(open('GBC_classifier_4datasets.sav', 'rb'))
yprob = loaded_clf.predict_proba(X_model_arr)
ypred = np.array(yprob[:,0]/(0.1*yprob[:,1]+(1-0.1)*yprob[:,2])<12.3).astype(int)

In [98]:
print('ypred: ', ypred)

ypred:  [0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0
 0 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 0 0 0 1 1
 0 1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0
 0 1 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 1 1 0 0 0 0 0 1 0 1]


Expert tree- generate label

In [99]:

from expert_tree import get_expert_tree_results, Expert_Tree

In [100]:
# generate 5 labels 
for i, row in data_df.iterrows():
    tag = Expert_Tree(row).run()

In [101]:
# generate 2 labels
labels_3, labelNames_3, flags_3 = get_expert_tree_results(data_df, three_class=False)

Evaluate

In [102]:
from sklearn.metrics import confusion_matrix
#t = confusion_matrix(expert_tree_result, ypred).ravel()
tn, fp, fn, tp = confusion_matrix(labels_3, ypred).ravel()
print(tn, fp, fn, tp)
validation_accuracy = (tn+tp)/(tn+tp+fn+fp)
sensitivity = tp/(fn+tp)
specificty= tn/(tn+fp)
print(validation_accuracy, sensitivity,specificty )

121 1 5 48
0.9657142857142857 0.9056603773584906 0.9918032786885246
